# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

In [2]:
#Importando o excel 
import pandas as pd

lacta=pd.read_excel('tweets_lacta.xlsx')

#Separando os tweets em tweets relevantes e irrelevantes
relevant_tweets=lacta[lacta['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante']==1]

irrelevant_tweets=lacta[lacta['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante']==0]

In [3]:
#Exceções - lista que contém elementos que devem ser retirados dos tweets para melhor análise 
excecoes=[':',';','[',']', '{','}','—','-','+','=','/','|', ')','_', '(','#','?','.', '!', ' ',',','“','”','\\\\','\n','\t','*','','"']

In [4]:
#Lista das palavras separadas de todos os tweets relevantes
lista_palavras_relevantes=[]

for tweet in relevant_tweets['Treinamento']:
    divide=tweet.split()
    for palavra in divide:
        if '@' not in palavra:
            if 'https' not in palavra:
                lista_letra=list(palavra)
                letras_corretas=[]
                for letra in lista_letra:
                    if letra not in excecoes:
                        letras_corretas.append(letra)
                palavra_limpa=''.join(letras_corretas)
                if palavra_limpa!='':
                    lista_palavras_relevantes.append(palavra_limpa)


In [5]:
#Lista das palavras separadas de todos os tweets irrelevantes
lista_palavras_irrelevantes=[]

for tweet in irrelevant_tweets['Treinamento']:
    divide=tweet.split()
    for palavra in divide:
        if '@' not in palavra:
            if 'https' not in palavra:
                lista_letra=list(palavra)
                letras_corretas=[]
                for letra in lista_letra:
                    if letra not in excecoes:
                        letras_corretas.append(letra)
                palavra_limpa=''.join(letras_corretas)
                if palavra_limpa!='':
                    lista_palavras_irrelevantes.append(palavra_limpa)


In [6]:
#Fazendo a contagem de cada palavra em relevantes e irrelevantes

palavras_relevantes={}

palavras_irrelevantes={}


for palavra in lista_palavras_relevantes:
    if palavra not in palavras_relevantes:
        palavras_relevantes[palavra]=2 #por conta do smoothing somou-se mais 1
    else:
        palavras_relevantes[palavra]+=1

for word in lista_palavras_irrelevantes:
    if word not in palavras_irrelevantes:
        palavras_irrelevantes[word]=2 #por conta do smoothing somou-se mais 1
    else:
        palavras_irrelevantes[word]+=1
        
#Total de palavras no excel de tweets relevantes
total_relev=sum(palavras_relevantes.values())

#Total de palavras no excel de tweets irrelevantes
total_irrelev=sum(palavras_irrelevantes.values())

#União das duas, sem repetição
lista_comum=list(palavras_relevantes.keys())+list(palavras_irrelevantes.keys())
lista_sem_rep=set(lista_comum)
length_lista=len(lista_sem_rep)

In [7]:
#Probabilidade de um tweet ser relevante ou não
p1=len(relevant_tweets)/len(lacta['Treinamento'])
p0=len(irrelevant_tweets)/len(lacta['Treinamento'])

#Encontrando a probabilidade de cada palavra
for palavra in palavras_relevantes:
    palavras_relevantes[palavra]=palavras_relevantes[palavra]/total_relev
    
for word in palavras_irrelevantes:
    palavras_irrelevantes[word]=palavras_irrelevantes[word]/total_irrelev
    
p1

0.11333333333333333

In [8]:
palavras_irrelevantes

{'$12': 0.0004097520999795124,
 '&lt3': 0.0004097520999795124,
 '08007041940': 0.0008195041999590248,
 '1': 0.0008195041999590248,
 '10': 0.0008195041999590248,
 '135': 0.0004097520999795124,
 '1380': 0.0004097520999795124,
 '16': 0.0004097520999795124,
 '17': 0.0004097520999795124,
 '1a': 0.0004097520999795124,
 '1kg': 0.0004097520999795124,
 '1º': 0.0004097520999795124,
 '2': 0.001229256299938537,
 '20': 0.0004097520999795124,
 '2º': 0.0004097520999795124,
 '3': 0.0014341323499282934,
 '3chocolate': 0.0004097520999795124,
 '3º': 0.0004097520999795124,
 '40%': 0.0006146281499692685,
 '44kilos': 0.0004097520999795124,
 '4h': 0.0004097520999795124,
 '4º': 0.0004097520999795124,
 '5': 0.0016390083999180496,
 '540': 0.0004097520999795124,
 '5star': 0.0016390083999180496,
 '5º': 0.0004097520999795124,
 '6': 0.001024380249948781,
 '6real': 0.0004097520999795124,
 '777': 0.0004097520999795124,
 'a': 0.011063306699446834,
 'abre': 0.0004097520999795124,
 'abrir': 0.0006146281499692685,
 'abri

In [9]:
#Criando a função para o classificador

def classificador(excel):
    classificacao=[]
    for tweet in excel['Teste']:
        quebra_tudo=[]
        divide=tweet.split()
        p_r=p1
        p_i=p0
        for palavra in divide:
            if '@' not in palavra:
                if 'https' not in palavra:
                    lista_letra=list(palavra)
                    letras_corretas=[]
                    for letra in lista_letra:
                        if letra not in excecoes:
                            letras_corretas.append(letra)
                    palavra_limpa=''.join(letras_corretas)
                    if palavra_limpa!='':
                        quebra_tudo.append(palavra_limpa)
        for item in quebra_tudo:
            if item not in palavras_relevantes.keys() and item in palavras_irrelevantes.keys():
                p_r*=1/(total_relev+length_lista)
                p_i*=palavras_irrelevantes[item]
            elif item in palavras_relevantes.keys() and item not in palavras_irrelevantes.keys():
                p_i*=1/(total_irrelev+length_lista)
                p_r*=palavras_relevantes[item]
            elif item in palavras_relevantes.keys() and item in palavras_irrelevantes.keys():
                p_r*=palavras_relevantes[item]
                p_i*=palavras_irrelevantes[item]
            else:
                p_r*=1/(total_relev+length_lista)
                p_i*=1/(total_irrelev+length_lista)
        if p_r>p_i:
            classi=1
        else:
            classi=0
        classificacao.append(classi)
    excel['Naive']=classificacao
    excel.to_excel('excel_classificado.xlsx')

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [10]:
teste = pd.read_excel('tweets_lacta.xlsx',sheet_name="Teste")
classificador(teste)

In [11]:
#Esse programa é utilizado para contar o acerto de nosso classificador e sua porcentagem.
a=pd.read_excel('excel_classificado.xlsx')
a
negativos=0
positivos=0
falsos_positivos=0
falsos_negativos=0
for n in range(len(a['Teste'])):
    if a['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante'][n]==1 and a['Naive'][n]==1:
        positivos+=1
    elif a['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante'][n]==1 and a['Naive'][n]==0:
        falsos_negativos+=1
    elif a['Classificador - 1 para relevante(críticas/sugestões) para a empresa e 0 para não relevante'][n]==0 and a['Naive'][n]==1:
        falsos_positivos+=1
    else:
        negativos+=1
print("Positivos: {0}. Sua porcentagem: {1}%".format(positivos,positivos*100/len(a['Teste'])))
print("Negativos: {0}. Sua porcentagem: {1}%".format(negativos,negativos*100/len(a['Teste'])))
print("Falsos-negativos: {0}. Sua porcentagem: {1}%".format(falsos_negativos,falsos_negativos*100/len(a['Teste'])))
print("Falsos-positivos: {0}. Sua porcentagem: {1}%".format(falsos_positivos,falsos_positivos*100/len(a['Teste'])))

Positivos: 26. Sua porcentagem: 13.0%
Negativos: 122. Sua porcentagem: 61.0%
Falsos-negativos: 4. Sua porcentagem: 2.0%
Falsos-positivos: 48. Sua porcentagem: 24.0%


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


       Observando o resultado do teste, o índice de acertos do classificador é razoável, com um índice de acerto de 74%. No entanto, analisando ainda a base de 'Treinamento' dos tweets, a probabilidade de um tweet ser relevante é cerca de 11%, enquanto a probabilidade do tweet não ser relevante é de 89%. Logo, caso o classificador retornasse sempre 0, isto é, retornar que todo tweet é irrelevante, o índice de acertos do classificador seria 89%. Portanto, comparando o índice real de acertos que obtivemos com o classificador, com o caso em que o classificador sempre devolve 0, o índice real de acertos ainda é baixo. Com base nessas informações, é bem provável que uma empresa não nos contrataria, pois ela esperaria um índice de acerto maior que 89%. No entanto, para aprimorar nosso classificador, podem ser feita algumas alterações. (cita as alterações. Pode ser em outro parágrafo)
       Analisando os dados um pouco mais a fundo, nota-se que a maior taxa de erro está no "Falsos-positivos" representando 24% do resultado dado pelo classificador, enquanto os "Falsos-negativos" representam 2%. Por outro lado, há certos aspectos que o classificador não leva em conta, como a dupla negação e o sarcasmo. Isso pode levar a uma futura iteração para o classificador. Para fazer com que o classificador detecte sarcasmo, podemos criar um banco de dados com tweets seguidos da hashtag "#sarcasmo" e aplicá-lo ao Naive Bayes (http://www.lewisgavin.co.uk/Sarcasm-Detector/). O classificador não poderia ser usado para gerar mais amostras pois se trata apenas de um programa que recebe um arquivo com vários tweets e retorna se os tweets são relevantes ou irrelevantes, mas não cria nenhum outro tweet. Caso ele gerasse amostras, ele pegaria palavras aleatórias do banco de dados e criaria um tweet sem sentido.
       Para aumentar a eficâcia do classificador, foram retiradas todas as palavras iniciadas por "http" pois indicavam links e são irrelevantes para a classificação. Outra iteração possível para a melhoria do classificador, seria implementar outros tipos de filtros, como por exemplo, retirar palavras de duas letras pois não mostram uma tendência clara de opinião. Por ter um banco de dados desbalanceaado e tendendo à irrelevante (89%), caso o programa fosse rodado chutando tudo irrelevante, deveria em torno 89% de acerto. O índice de acerto retornado é razoável, porém, não excede 89%. As chances da empresa continuar financiando o projeto seriam menores do que se tivesse excedido 89%. Para melhorar essa taxa, poderíamos aumentar a quantidade de tweet relevantes no banco de dados para de um balançeada e aumentar o acerto do classificador.
       O classificador Naive Bayes poderia ser utilizado também para desenvolvimento de IA. Seria possível fazer uma pergunta para a IA, ela passaria pelo processo de classificação e retornaria uma resposta à pergunta. Ou então para classificar um objeto. O usuário entra com o formato e as dimensões do objeto e o classificador retorna qual objeto é.